In [ ]:
from ipyleaflet import WidgetControl
from component.tiles import AlertMap, PanelTile
from component.model import AlertModel
import sepal_ui.sepalwidgets as sw

In [ ]:
model = AlertModel()

In [ ]:
map_ = AlertMap(model)

In [ ]:
panel_tile = PanelTile(map_, model)

In [ ]:
control_panel = WidgetControl(
    widget=panel_tile, 
    position='bottomright', 
    transparent_bg=True
)
map_.controls  = tuple(
    [control_panel] + [c for c in map_.controls]
)
map_.layout.height="100vh"

In [ ]:
process_tile = sw.Tile(id_='ui', title='', inputs=[map_])

appBar = sw.AppBar(title='Sepal Planet active fires explorer (SEPAFE)')

content = [
    process_tile,
]

#create a drawer 
item_process = sw.DrawerItem(
    'Map', 
    'mdi-map-marker-check', 
    card="ui"
).display_tile(content)

code_link = 'https://github.com/ingdanielguerrero/planet_active_fires_explorer'
wiki_link = 'https://github.com/dfguerrerom/planet_active_fires_explorer/blob/main/doc/en.rst'
issue = 'https://github.com/ingdanielguerrero/planet_active_fires_explorer/issues/new'

items = [
    item_process,
]

drawer = sw.NavDrawer(
    items, 
    code = code_link, 
    wiki = wiki_link, 
    issue = issue, 
    mini_variant=True
).display_drawer(appBar.toggle_button)

#build the app 
app = sw.App(
    appBar = appBar,
    tiles=content, 
    navDrawer=drawer
).show_tile('ui')
#display the app
app